# Engineer binders based on PDB structures 

In [1]:
%pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 33.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 77.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from Bio.PDB import PDBParser, NeighborSearch, Selection
from Bio.SeqUtils import seq1
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio import SeqIO 
import random


def parse_pdb_file(pdb_file):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('structure', pdb_file)
    pdb_filename = os.path.basename(pdb_file)
    pdb_id = os.path.splitext(pdb_filename)[0]
    return structure, pdb_id


def get_target_residues(chain, start_residue, end_residue):
    return [residue for residue in chain if start_residue <= residue.get_id()[1] <= end_residue]


def find_neighbor_residues(target_atoms, neighbor_chain, distance_threshold):
    ns = NeighborSearch(list(neighbor_chain.get_atoms()))
    neighbor_residues = set()
    for atom in target_atoms:
        neighbors = ns.search(atom.get_coord(), distance_threshold, level='R')
        neighbor_residues.update(neighbors)
    return neighbor_residues


def extract_proximal_sequence(pdb_file, chain_id, target_residues_range, neighbor_chain_id, N, max_distance, increment):
    structure, pdb_id = parse_pdb_file(pdb_file)
    target_chain = structure[0][chain_id]
    neighbor_chain = structure[0][neighbor_chain_id]
    
    start_residue, end_residue = target_residues_range
    target_residues = get_target_residues(target_chain, start_residue, end_residue)
    target_atoms = Selection.unfold_entities(target_residues, 'A')
    
    distance_threshold = random.uniform(1.0, max_distance / 2)
    neighbor_resids_list = []
    
    while distance_threshold <= max_distance:
        neighbor_residues = find_neighbor_residues(target_atoms, neighbor_chain, distance_threshold)
        neighbor_residues -= set(target_residues)
        neighbor_resids = sorted(set(residue.get_id() for residue in neighbor_residues))
        
        if neighbor_resids:
            resseq_ranges = get_contiguous_residue_ids(neighbor_resids)
            neighbor_resids_list.extend(resseq_ranges)
        
        distance_threshold += increment
    
    if neighbor_resids_list:
        selected_range = random.choice(neighbor_resids_list)
        min_residue_id, max_residue_id = selected_range
        sequence_residues = [
            residue for residue in neighbor_chain
            if min_residue_id <= residue.get_id() <= max_residue_id
        ]
        
        # Adjust sequence length to N
        sequence_residues = adjust_sequence_length(sequence_residues, neighbor_chain, N)
        if sequence_residues is None:
            return None
        
        # Create sequence record
        return create_sequence_record(neighbor_chain, sequence_residues, pdb_id, neighbor_chain_id)
    return None


def create_sequence_record(neighbor_chain, sequence_residues, pdb_id, neighbor_chain_id):
    sequence = ''.join([seq1(residue.get_resname()) for residue in sequence_residues])
    residue_ids = [residue.get_id() for residue in sequence_residues]
    
    min_residue_id = residue_ids[0]
    max_residue_id = residue_ids[-1]
    
    min_resseq, min_icode = min_residue_id[1], min_residue_id[2].strip()
    max_resseq, max_icode = max_residue_id[1], max_residue_id[2].strip()
    
    # Format insertion codes if they are present
    min_icode = min_icode if min_icode else ''
    max_icode = max_icode if max_icode else ''
    
    seq_id = f"{pdb_id}_chain{neighbor_chain_id}_resseq_{min_resseq}{min_icode}-{max_resseq}{max_icode}"
    return SeqRecord(Seq(sequence), id=seq_id, description="")


def get_contiguous_residue_ids(residue_ids):
    ranges = []
    start = residue_ids[0]
    prev = residue_ids[0]
    for residue_id in residue_ids[1:]:
        if residue_id[1] == prev[1] + 1 and residue_id[2] == ' ':
            prev = residue_id
        else:
            ranges.append((start, prev))
            start = residue_id
            prev = residue_id
    ranges.append((start, prev))
    return ranges


def adjust_sequence_length(sequence_residues, neighbor_chain, N):
    if len(sequence_residues) == N:
        return sequence_residues
    elif len(sequence_residues) > N:
        excess = len(sequence_residues) - N
        start = random.randint(0, excess)
        return sequence_residues[start:start + N]
    
    extended_sequence = extend_sequence(sequence_residues, neighbor_chain, N)
    if extended_sequence is None:
        return None
    else:
        return extended_sequence


def extend_sequence(sequence_residues, neighbor_chain, N):
    residues_needed = N - len(sequence_residues)
    residue_ids = [residue.get_id() for residue in sequence_residues]
    min_residue_id = min(residue_ids)
    max_residue_id = max(residue_ids)
    
    left_residues = [residue for residue in neighbor_chain if residue.get_id() < min_residue_id]
    right_residues = [residue for residue in neighbor_chain if residue.get_id() > max_residue_id]
    
    left_residues.sort(key=lambda r: r.get_id(), reverse=True)
    right_residues.sort(key=lambda r: r.get_id())
    
    added_residues = []
    
    while residues_needed > 0 and (left_residues or right_residues):
        if left_residues and right_residues:
            side = random.choice(['left', 'right'])
        elif left_residues:
            side = 'left'
        elif right_residues:
            side = 'right'
        else:
            break
        
        if side == 'left' and left_residues:
            added_residues.insert(0, left_residues.pop(0))
            residues_needed -= 1
        elif side == 'right' and right_residues:
            added_residues.append(right_residues.pop(0))
            residues_needed -= 1
    
    final_sequence = added_residues + sequence_residues
    if len(final_sequence) == N:
        return final_sequence
    elif len(final_sequence) > N:
        return final_sequence[:N]
    else:
        return None


def main():
    pdb_ids = ['6vja', '6y92', '6y97']
    chain_ids = ['D', 'B', 'A']
    neighbor_chain_ids = ['I', 'D', 'L']
    target_residues_ranges = [(166, 183), (161, 183), (166, 183)]
    # pdb_id = '6vja'
    # chain_id = 'D'
    # neighbor_chain_id = 'I'
    # target_residues_range = (166, 183)

    for pdb_id,chain_id,neighbor_chain_id,target_residues_range in zip(pdb_ids,chain_ids,neighbor_chain_ids,target_residues_ranges):
    
        pdb_file = f'../../data/structures/{pdb_id}.pdb'
        output_fasta = f'../../results/predictions/{pdb_id}_chain{neighbor_chain_id}_rational_designs.fasta'
        
        N = 80
        max_distance = 20.0
        increment = 0.5
        num_seq = 100
        
        seq_records = []
        unique_sequences = set()
        attempts = 0
        max_attempts = 1000
        
        while len(seq_records) < num_seq and attempts < max_attempts:
            seq_record = extract_proximal_sequence(pdb_file, chain_id, target_residues_range, neighbor_chain_id, N, max_distance, increment)
            attempts += 1
            if seq_record:
                sequence_str = str(seq_record.seq)
                seq_id = seq_record.id
                if sequence_str not in unique_sequences and seq_id not in [rec.id for rec in seq_records]:
                    unique_sequences.add(sequence_str)
                    seq_records.append(seq_record)
        
        SeqIO.write(seq_records, output_fasta, 'fasta')
        print(f"{len(seq_records)} unique sequences saved to {output_fasta}")


if __name__ == '__main__':
    main()


100 unique sequences saved to ../../results/predictions/6vja_chainI_rational_designs.fasta
100 unique sequences saved to ../../results/predictions/6y92_chainD_rational_designs.fasta
100 unique sequences saved to ../../results/predictions/6y97_chainL_rational_designs.fasta


In [72]:
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq1

def get_fragseq(pdb_file,
                chain_id,
                resseq_range):

    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('structure',pdb_file)

    chain = structure[0][chain_id]

    residues = list(chain)
    left,right = resseq_range
    sequence_residues = residues[left:right]
    sequence = ''.join([seq1(residue.get_resname()) for residue in sequence_residues])

    return sequence

print('Chain I, 1-80: ',get_fragseq(pdb_file, 'I', (0,80)), len(get_fragseq(pdb_file, 'I', (0,80))))
print('Chain D, 166-184: ',get_fragseq(pdb_file, 'D', (166-46,184-46)), len(get_fragseq(pdb_file, 'D', (166-46,184-46))))


Chain I, 1-80:  QVQLQQPGAELVKPGASVKMSCKASGYTFTSYNMHWVKQTPGRGLEWIGAIYPGNGDTSYNQKFKGKATLTADKSSSTAY 80
Chain D, 166-184:  NCEPANPSEKNSPSTQYC 18
